# Neo4j

Image: https://hub.docker.com/_/neo4j

docker run \
    --publish=7474:7474 --publish=7687:7687 \
    --volume=$HOME/neo4j/data:/data
    --env=NEO4J_AUTH=none \
    neo4j

Relationship with props: https://pythontic.com/database/neo4j/create%20nodes%20and%20relationships 

Handbook: https://neo4j-contrib.github.io/py2neo/index.html#guides

GUI (web): http://localhost:7474/browser/

Clear DB:

```
MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n,r
```

In [92]:
import json
from elasticsearch import Elasticsearch
from py2neo import Graph, Node, Relationship

In [93]:
client = Elasticsearch([{"host": "127.0.0.1", "scheme": "http", "port": 9200}])

client.ping()

True

In [94]:
graph_db = Graph("bolt://localhost:7687")

In [95]:
searchBody = {
  "size": 9999,
  "_source": True,
  "query": {
    "match_all": {}
  }
}

result = client.search(index="master", body=searchBody)

In [96]:
result['hits']['total']

{'value': 20, 'relation': 'eq'}

In [97]:
masters = result['hits']['hits']

In [98]:
len(masters)

20

In [99]:
masters[:3]

[{'_index': 'master',
  '_id': '618529',
  '_score': 1.0,
  '_source': {'master_id': 618529,
   'master_desc': 'Аполлон Харитонович Афанасьев, Стаж Работы: 5 Л./Г..',
   'master_feedbacks': ['медлительный, аккуратный.',
    'ворчливый, медлительный.']}},
 {'_index': 'master',
  '_id': '791839',
  '_score': 1.0,
  '_source': {'master_id': 791839,
   'master_desc': 'Август Дорофеевич Кошелев, Стаж Работы: 11 Л./Г..',
   'master_feedbacks': ['медлительный, ворчливый.']}},
 {'_index': 'master',
  '_id': '800991',
  '_score': 1.0,
  '_source': {'master_id': 800991,
   'master_desc': 'Сысоев Селиверст Елизарович, Тип Образования: Высшее.',
   'master_feedbacks': ['аккуратный, ворчливый.', 'ворчливый, аккуратный.']}}]

In [100]:
# searchBody = {
#   "_source": True,
#   "query": {
#     "term": {
#       "order_master_id": 800991
#    }
#   }
# }

# result = client.search(index="order", body=searchBody)

In [101]:
# result['hits']

In [104]:
i = 0
for master in masters:
    try:
         print(f'{i})', end='')
         MasterNode = Node("Master",master_id=master['_source']['master_id'], master_desc=master['_source']['master_desc'])
         graph_db.create(MasterNode)
         searchBody = {
              "size": 9999,
              "_source": True,
              "query": {
                "term": {
                  "order_master_id": master['_source']['master_id']
               }
              }
            }
         result = client.search(index="order", body=searchBody)
         orders = result['hits']['hits']
         for order in orders:
             OrderNode = Node("Order",order_id=order['_source']['order_id'], order_date=order['_source']['order_date'], order_customer_desc=order['_source']['order_customer_desc'], order_price=order['_source']['order_price'])
             graph_db.create(OrderNode)
             NodesRelationship = Relationship(MasterNode, "Executed", OrderNode, order_due_date=order['_source']['order_due_date'], order_fact_completion_date=order['_source']['order_fact_completion_date'])
             graph_db.create(NodesRelationship)
             print('.', end='')
         print("+")
         i += 1
    except Exception as e:
        print(f'Exception: {e}')
        continue

0)......+
1)..+
2)......+
3).......+
4)...+
5)..........+
6)....+
7)........+
8).......+
9)........+
10)...+
11).....+
12)...+
13).....+
14)...+
15)........+
16)+
17)..+
18).....+
19).....+
